(mmt-tune)=

# Batch training & tuning on Ray Tune

**Batch training** and tuning are common tasks in simple machine learning use-cases such as time series forecasting. They require fitting of simple models on data batches corresponding to different locations, products, etc. Batch training can take less time to process all the data at once, but only if those batches can run in parallel!

This notebook showcases how to conduct batch training regression algorithms from [XGBoost](https://docs.ray.io/en/latest/tune/examples/tune-xgboost.html) and [Scikit-learn](https://docs.ray.io/en/latest/ray-more-libs/joblib.html) with **[Ray Tune](https://docs.ray.io/en/latest/tune/index.html)**. **XGBoost** is a popular open-source library used for regression and classification. **Scikit-learn** is a popular open-source library with a vast assortment of well-known ML algorithms.

```{tip}
The workload showcased in this notebook can be expressed using different Ray components, such as Ray Data, Ray Tune and Ray Core.
For best practices, see {ref}`ref-use-cases-mmt`.
```

![Batch training diagram](../../data/examples/images/batch-training.svg)

For the data, we will use the [NYC Taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). This popular tabular dataset contains historical taxi pickups by timestamp and location in NYC.

For the training, we will train separate regression models to predict `trip_duration`, with a different model for each dropoff location in NYC. Specifically, we will conduct an experiment for each `dropoff_location_id`, to find the best either XGBoost or Scikit-learn model, per location.

# Contents

In this this tutorial, you will learn about:
 1. [Define how to load and prepare Parquet data](#load_data)
 2. [Define your Ray Tune Search Space and Search Algorithm](#define_search_space)
 3. [Define a Trainable (callable) function](#define_trainable)
 4. [Run batch training on Ray Tune](#run_tune_search)
 5. [Load a model from checkpoint and perform batch prediction](#load_checkpoint)


# Walkthrough

```{tip}
Prerequisite for this notebook: Read the [Key Concepts](https://docs.ray.io/en/latest/tune/key-concepts.html) page for Ray Tune.
```

Let us start by importing a few required libraries, including open-source [Ray](https://github.com/ray-project/ray) itself!

In [1]:
import os

print(f"Number of CPUs in this system: {os.cpu_count()}")
from typing import Tuple, List, Union, Optional, Callable
import time
import pandas as pd
import numpy as np

print(f"numpy: {np.__version__}")
import pyarrow
import pyarrow.parquet as pq
import pyarrow.dataset as pds

print(f"pyarrow: {pyarrow.__version__}")

Number of CPUs in this system: 8
numpy: 1.21.6
pyarrow: 10.0.0


In [2]:
import ray

if ray.is_initialized():
    ray.shutdown()
ray.init()

Python version:,3.8.13
Ray version:,2.1.0
Dashboard:,http://127.0.0.1:8266


In [3]:
print(ray.cluster_resources())

{'object_store_memory': 2147483648.0, 'memory': 8666939392.0, 'CPU': 8.0, 'node:127.0.0.1': 1.0}


In [4]:
# import standard sklearn libraries
import sklearn
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

print(f"sklearn: {sklearn.__version__}")
import xgboost as xgb

print(f"xgboost: {xgb.__version__}")
# import ray libraries
from ray import air, tune
from ray.air import session
from ray.air.checkpoint import Checkpoint

# set global random seed for sklearn models
np.random.seed(415)

sklearn: 1.1.3
xgboost: 1.7.1


In [5]:
# For benchmarking purposes, we can print the times of various operations.
# In order to reduce clutter in the output, this is set to False by default.
PRINT_TIMES = False


def print_time(msg: str):
    if PRINT_TIMES:
        print(msg)


# To speed things up, we’ll only use a small subset of the full dataset consisting of two last months of 2019.
# You can choose to use the full dataset for 2018-2019 by setting the SMOKE_TEST variable to False.
SMOKE_TEST = True

## Define how to load and prepare Parquet data <a class="anchor" id="load_data"></a>

First, we need to load some data. Since the NYC Taxi dataset is fairly large, we will filter files first into a PyArrow dataset. And then in the next cell after, we will filter the data on read into a PyArrow table and convert that to a pandas dataframe.

```{tip}
Use PyArrow dataset and table for reading or writing large parquet files, since its native multithreaded C++ adapter is faster than pandas read_parquet, even using engine=pyarrow.
```

In [6]:
# Define some global variables.
TARGET = "trip_duration"
s3_partitions = pds.dataset(
    "s3://anonymous@air-example-data/ursa-labs-taxi-data/by_year/",
    partitioning=["year", "month"],
)
s3_files = [f"s3://anonymous@{file}" for file in s3_partitions.files]

# Obtain all location IDs
all_location_ids = (
    pq.read_table(s3_files[0], columns=["dropoff_location_id"])[
        "dropoff_location_id"
    ]
    .unique()
    .to_pylist()
)
# drop [264, 265]
all_location_ids.remove(264)
all_location_ids.remove(265)

# Use smoke testing or not.
starting_idx = -1 if SMOKE_TEST else 0
# TODO: drop location 199 to test error-handling before final git checkin
sample_locations = [141, 229, 173] if SMOKE_TEST else all_location_ids

# Display what data will be used.
s3_files = s3_files[starting_idx:]
print(f"NYC Taxi using {len(s3_files)} file(s)!")
print(f"s3_files: {s3_files}")
print(f"Locations: {sample_locations}")

NYC Taxi using 1 file(s)!
s3_files: ['s3://anonymous@air-example-data/ursa-labs-taxi-data/by_year/2019/06/data.parquet/ab5b9d2b8cc94be19346e260b543ec35_000000.parquet']
Locations: [141, 229, 173]


In [7]:
# Function to read a pyarrow.Table object using pyarrow parquet
def read_data(file: str, sample_id: np.int32) -> pd.DataFrame:

    df = pq.read_table(
        file,
        filters=[
            ("passenger_count", ">", 0),
            ("trip_distance", ">", 0),
            ("fare_amount", ">", 0),
            ("pickup_location_id", "not in", [264, 265]),
            ("dropoff_location_id", "not in", [264, 265]),
            ("dropoff_location_id", "=", sample_id),
        ],
        columns=[
            "pickup_at",
            "dropoff_at",
            "pickup_location_id",
            "dropoff_location_id",
            "passenger_count",
            "trip_distance",
            "fare_amount",
        ],
    ).to_pandas()

    return df


# Function to transform a pandas dataframe
def transform_df(input_df: pd.DataFrame) -> pd.DataFrame:
    df = input_df.copy()

    # calculate trip_duration
    df["trip_duration"] = (df["dropoff_at"] - df["pickup_at"]).dt.seconds
    # filter trip_durations > 1 minute and less than 24 hours
    df = df[df["trip_duration"] > 60]
    df = df[df["trip_duration"] < 24 * 60 * 60]
    # keep only necessary columns
    df.drop(
        ["dropoff_at", "pickup_at", "pickup_location_id", "fare_amount"],
        axis=1,
        inplace=True,
    )
    df["dropoff_location_id"] = df["dropoff_location_id"].fillna(-1)
    return df

## Define your Ray Tune Search Space and Search Algorithm <a class="anchor" id="define_search_space"></a>

In this notebook, we will use Ray Tune to run parallel training jobs per dropoff location. The training jobs will be defined using a search space and simple grid search. Depending on your need, fancier search spaces and search algorithms are possible with Tune. 

**First, define a search space of experiment trials to run.**  
> The typical use case for Tune search spaces are for hypterparameter tuning. In our case, we are defining a Tune search space in a way to allow for training jobs to be conducted automatically. Each training job will run on a different data partition (taxi dropoff location) and use a different algorithm. 

**Next, define a search algorithm.**

```{tip}
Common search algorithms include grid search, random search, and Bayesian optimization. For more details, see [Working with Tune Search Spaces](https://docs.ray.io/en/master/tune/tutorials/tune-search-spaces.html#tune-search-space-tutorial). Deciding the best combination of search space and search algorithm is part of the art of being a Data Scientist and depends on the data, algorithm, and problem being solved.
```

Ray Tune will use the search space and the specified search algorithm to generate multiple configurations, each of which will be evaluated in a separate Trial on a Ray Cluster. Ray Tune will take care of orchestrating those Trials automatically. Specifically, Ray Tune will pass a config dictionary to each partition and make a Trainable function call.

**Below, we define our search space consists of:**
- Different algorithms:
  - XGBoost
  - Scikit-learn LinearRegression
- Some or all NYC taxi drop-off locations. 

For Tune search algorithm, we want to run *grid search*, meaning we want to run an experiment for every possible combination in the search space. What this means is every algorithm will be applied to every NYC Taxi drop-off location.

In [8]:
# 1. Define a search space.
search_space = {
    "algorithm": tune.grid_search(
        [LinearRegression(fit_intercept=True), xgb.XGBRegressor(max_depth=4)]
    ),
    "location": tune.grid_search(sample_locations),
}

## Define a Trainable (callable) function <a class="anchor" id="define_trainable"></a>

📈 Typically when you are running Data Science experiments, you want to be able to keep track of summary metrics for each trial, so you can decide at the end which trials were best. That way, you can decide which model to deploy.

🇫 Next, we define a trainable function in order to train and evaluate a scikit-learn model on a data partition. This function will be called in parallel by every Tune trial. Inside this trainable function, we will:
- Add detailed metrics we want to report (each model's loss or error). 
- Checkpoint each model for easy deployment later.

📖 **The metrics defined inside the trainable function will appear in the Ray Tune experiment summary table.**
```{tip}
Ray Tune has two ways of defining a trainable, namely the [Function API](https://docs.ray.io/en/latest/tune/api_docs/trainable.html#trainable-docs) and the Class API. Both are valid ways of defining a trainable, but *the Function API is generally recommended*.
```

**In the cell below, we define a "Trainable" function called `train_model()`**.  
- The input is a config dictionary argument. 
- The output can be a simple dictionary of metrics which will be reported back to Tune. 
- We will [checkpoint](https://docs.ray.io/en/master/ray-air/key-concepts.html#checkpoints) save each model in addition to reporting each trial's metrics.
  > For checkpointing, we use `ray.air.checkpoint.Checkpoint`. *Ray AIR includes integrations to popular ML libraries, including Scikit-learn*. This makes it possible to use the convenient AIR API abstractions, without having to specify code details of the Scikit-learn library itself.
- Since we are using **grid search**, this means `train_model()` will be run *in parallel for every permutation* in the Tune search space!

In [9]:
from ray.air import Checkpoint

# 2. Define a custom train function
def train_model(config: dict) -> None:

    algorithm = config["algorithm"]
    sample_location_id = config["location"]

    # Load data.
    df_list = [read_data(f, sample_location_id) for f in s3_files]
    df_raw = pd.concat(df_list, ignore_index=True)

    # Transform data.
    df = transform_df(df_raw)

    # We need at least 10 rows to create a train / test split.
    if df.shape[0] < 10:
        print_time(
            f"Location {sample_location_id} has only {df.shape[0]} rows."
        )
        session.report(dict(error=None))
        return None

    # Train/valid split.
    train_df, valid_df = train_test_split(df, test_size=0.2, shuffle=True)
    train_X = train_df[["passenger_count", "trip_distance"]]
    train_y = train_df[TARGET]
    valid_X = valid_df[["passenger_count", "trip_distance"]]
    valid_y = valid_df[TARGET]

    # Train model.
    model = algorithm.fit(train_X, train_y)
    pred_y = model.predict(valid_X)

    # Evaluate.
    error = sklearn.metrics.mean_absolute_error(valid_y, pred_y)

    # Define a model checkpoint using AIR API.
    # https://docs.ray.io/en/latest/tune/tutorials/tune-checkpoints.html
    checkpoint = ray.air.checkpoint.Checkpoint.from_dict(
        {"model": algorithm, "location_id": sample_location_id}
    )

    # Save checkpoint and report back metrics, using ray.air.session.report()
    # The metrics you specify here will appear in Tune summary table.
    # They will also be recorded in Tune results under `metrics`.
    metrics = dict(error=error)
    session.report(metrics, checkpoint=checkpoint)

## Run batch training on Ray Tune <a class="anchor" id="run_tune_search"></a>


**Now we are ready to kick off a Ray Tune experiment!**  

Recall what we are doing, high level, is training several different models per dropoff location. We are using Ray Tune so we can run all these trials in parallel. At the end, we will inspect the results of the experiment and deploy only the best model per dropoff location.

**In the cell below, we use AIR configs and run the experiment using `tuner.fit()`.** 

Tune will report on experiment status, and after the experiment finishes, you can inspect the results. 

- In the cell below, we use the default resources config which is 1 CPU core for each task. For more information about configuring resource allocations, see [A Guide To Parallelism and Resources](https://docs.ray.io/en/master/tune/tutorials/tune-resources.html#tune-parallelism). 

- In the AIR config below, we have specified a local directory `my_Tune_logs` for logging instead of the default `~/ray_results` directory. Giving your logs a project name makes them easier to find. Also giving a relative path, means you can see your logs inside the Jupyter browser. Learn more about logging Tune results at [How to configure logging in Tune](https://docs.ray.io/en/master/tune/tutorials/tune-output.html#tune-logging).

- Tune can [retry failed experiments automatically](https://docs.ray.io/en/master/tune/tutorials/tune-stopping.html#tune-stopping-guide), as well as entire experiments. This is necessary in case a node on your remote cluster fails (when running on a cloud such as AWS or GCP).

💡 Right-click on the cell below and choose "Enable Scrolling for Outputs"! This will make it easier to view, since model training output can be very long!

**Setting SMOKE_TEST=False, running on Anyscale:  518 models, using 18 NYC Taxi S3 files dating from 2018/01 to 2019/06 (split into partitions approx 1GiB each), simultaneously trained on a 23-node AWS cluster of [m5.4xlarges](https://aws.amazon.com/ec2/instance-types/m5/). Total data reading and train time was 37 minutes.**

In [10]:
# By default, Tune reserves 1 CPU core per task.
# # 3. Customize resources per trial, here we set 1 CPU each.
# train_model = tune.with_resources(train_model, {"cpu": 1})

# Define a tuner object using Ray AIR Tuner API
tuner = tune.Tuner(
    train_model,
    param_space=search_space,
    run_config=air.RunConfig(
        # redirect logs to relative path instead of default ~/ray_results/
        local_dir="my_Tune_logs",
        name="batch_tuning",
        # Set Ray Tune verbosity. Print summary table only with levels 2 or 3.
        verbose=2,
    ),
)

# 4. Run the experiment with Ray Tune
start = time.time()
results = tuner.fit()
total_time_taken = time.time() - start

# Print some training stats
print(f"Total number of models: {len(results)}")
print(f"TOTAL TIME TAKEN: {total_time_taken:.2f} seconds")
best_result = results.get_best_result(metric="error", mode="min").config
print(f"Best result: {best_result}")

Trial name,error,should_checkpoint
train_model_6b248_00000,509.049,True
train_model_6b248_00001,505.486,True
train_model_6b248_00002,534.634,True
train_model_6b248_00003,584.448,True
train_model_6b248_00004,1776.74,True
train_model_6b248_00005,1616.84,True


2022-12-06 14:44:44,655	INFO tune.py:777 -- Total run time: 69.12 seconds (68.37 seconds for the tuning loop).


Total number of models: 6
TOTAL TIME TAKEN: 69.15 seconds
Best result: {'algorithm': XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=4, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...), 'location': 141}


<br>

**After the Tune experiment has run, select the best model per dropoff location.**

We can assemble the Tune results ([ResultGrid object](https://docs.ray.io/en/master/tune/examples/tune_analyze_results.html)) into a pandas dataframe, then sort by minimum error, to select the best model per dropoff location.

In [11]:
# get a list of training loss errors
errors = [i.metrics.get("error", 10000.0) for i in results]

# get a list of checkpoints
checkpoints = [i.checkpoint for i in results]

# get a list of locations
locations = [i.config["location"] for i in results]

# get a list of model params
algorithms = [i.config["algorithm"] for i in results]

# Assemble a pandas dataframe from Tune results
results_df = pd.DataFrame(
    zip(locations, errors, algorithms, checkpoints),
    columns=["location_id", "error", "algorithm", "checkpoint"],
)
print(results_df.dtypes)
results_df.head(8)

location_id      int64
error          float64
algorithm       object
checkpoint      object
dtype: object


,location_id,error,algorithm,checkpoint
0,141,509.049035,LinearRegression(),Checkpoint(local_path=/Users/christy/Documents...
1,141,505.486459,"XGBRegressor(base_score=0.5, booster='gbtree',...",Checkpoint(local_path=/Users/christy/Documents...
2,229,534.633924,LinearRegression(),Checkpoint(local_path=/Users/christy/Documents...
3,229,584.447774,"XGBRegressor(base_score=0.5, booster='gbtree',...",Checkpoint(local_path=/Users/christy/Documents...
4,173,1776.735955,LinearRegression(),Checkpoint(local_path=/Users/christy/Documents...
5,173,1616.835263,"XGBRegressor(base_score=0.5, booster='gbtree',...",Checkpoint(local_path=/Users/christy/Documents...


In [12]:
# Keep only 1 model per location_id with minimum error
final_df = results_df.copy()
final_df = final_df.loc[(final_df.error > 0), :]
final_df = final_df.loc[final_df.groupby("location_id")["error"].idxmin()]
final_df.sort_values(by=["error"], inplace=True)
final_df.set_index("location_id", inplace=True, drop=True)
print(final_df.dtypes)
final_df

error         float64
algorithm      object
checkpoint     object
dtype: object


,error,algorithm,checkpoint
location_id,,,
141,505.486459,"XGBRegressor(base_score=0.5, booster='gbtree',...",Checkpoint(local_path=/Users/christy/Documents...
229,534.633924,LinearRegression(),Checkpoint(local_path=/Users/christy/Documents...
173,1616.835263,"XGBRegressor(base_score=0.5, booster='gbtree',...",Checkpoint(local_path=/Users/christy/Documents...


In [13]:
final_df[["algorithm"]].astype("str").value_counts(normalize=True)

algorithm                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,\n             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,\n             early_stopping_rounds=None, enable_categorical=False,\n             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,\n        

## Load a model from checkpoint and perform batch prediction  <a class="anchor" id="load_checkpoint"></a>

```{tip}
[Ray AIR Predictors](https://docs.ray.io/en/latest/ray-air/predictors.html) make batch inference easy since they have internal logic to parallelize the inference.
```

Finally, we will restore the best and worst models from checkpoint and make predictions. 

- We will easily obtain AIR Checkpoint objects from the Tune results. 
- We will restore a regression model directly from checkpoint, and demonstrate it can be used for prediction.

In [14]:
# Choose a dropoff location
sample_location_id = final_df.index[0]
sample_location_id

141

In [15]:
# Get the algorithm used
sample_algorithm = final_df.loc[[sample_location_id]].algorithm.values[0]
print(f"algorithm type:: {type(sample_algorithm)}")

# Get a checkpoint directly from the pandas dataframe of Tune results
checkpoint = final_df.checkpoint[sample_location_id]
print(f"checkpoint type:: {type(checkpoint)}")

# Restore a model from checkpoint
sample_model = checkpoint.to_dict()["model"]

algorithm type:: <class 'xgboost.sklearn.XGBRegressor'>
checkpoint type:: <class 'ray.air.checkpoint.Checkpoint'>


In [16]:
# Create some test data
df_list = [read_data(f, sample_location_id) for f in s3_files[:1]]
df_raw = pd.concat(df_list, ignore_index=True)
df = transform_df(df_raw)

# Train/test split.
_, test_df = train_test_split(df, test_size=0.2, shuffle=True)
test_X = test_df[["passenger_count", "trip_distance"]]
test_y = np.array(test_df.trip_duration)  # actual values

In [17]:
# Perform batch prediction using restored model from checkpoint
pred_y = sample_model.predict(test_X)

# Zip together predictions and actuals to visualize
pd.DataFrame(zip(pred_y, test_y), columns=["pred_y", "trip_duration"])[0:10]

,pred_y,trip_duration
0,1370.029419,1174
1,602.880676,299
2,1261.975098,1206
3,322.560333,566
4,712.604370,630
5,1430.202637,852
6,1618.045898,1596
7,606.959839,801
8,8332.479492,1363
9,1021.077026,715


**Compare validation and test error.**

During model training we reported error on "validation" data (random sample). Below, we will report error on a pretend "test" data set (a different random sample).

Do a quick validation that both errors are reasonably close together.

In [18]:
# Evaluate restored model on test data.
error = sklearn.metrics.mean_absolute_error(test_y, pred_y)
print(f"Test error: {error}")

Test error: 498.45033775758844


In [19]:
# Compare test error with training validation error
print(f"Validation error: {final_df.error[sample_location_id]}")

# Validation and test errors should be reasonably close together.

Validation error: 505.4864588184019
